In [1]:
# Import dependencies
import pandas as pd
import requests
from io import StringIO
from bs4 import BeautifulSoup

In [1]:
company = 'RelianceIndustries'

# Url Dictionary
links = {
    'Balance Sheet': 'https://www.moneycontrol.com/financials/sbi/balance-sheetVI/RI#RI',
    'Profit Loss': 'https://www.moneycontrol.com/financials/sbi/profit-lossVI/RI#RI',
    'Quarterly Results': 'https://www.moneycontrol.com/financials/sbi/results/quarterly-results/RI#RI',
    'Half Yearly Results': 'https://www.moneycontrol.com/financials/sbi/results/half-yearly/RI#RI',
    'Nine Months Results': 'https://www.moneycontrol.com/financials/sbi/results/nine-months/RI#RI',
    'Yearly Results': 'https://www.moneycontrol.com/financials/sbi/results/yearly/RI#RI',
    'Cash Flow': 'https://www.moneycontrol.com/financials/sbi/cash-flowVI/RI#RI',
    'Ratios': 'https://www.moneycontrol.com/financials/sbi/ratiosVI/RI#RI',
}
# Capital_Structure = 'https://www.moneycontrol.com/financials/relianceindustries/capital-structure/RI#RI'

In [4]:
def get_financials(links_dict, company='company'):
    """
    Fetch financial data from URLs, save them to an Excel file, and return the file path.

    Parameters:
    - links_dict (dict): A dictionary with sheet names as keys and URLs as values.
    - company (str): The name of the company, used to generate the output file name.

    Returns:
    - str: The file path of the saved Excel file.
    """
    # Create the output file name
    output_file = f'{company}_Financials.xlsx'

    # Write data to the Excel file
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        for key, url in links_dict.items():
            print(f"Processing {key}...")

            # Get HTML data
            response = requests.get(url)
            response.raise_for_status()
            html_data = response.content

            # Parse the HTML
            soup = BeautifulSoup(html_data, 'html.parser')

            # Get the table
            table_html = soup.find('table', class_='mctable1')
            if not table_html:
                print(f"No table found for {key}. Skipping...")
                continue

            # Convert to DataFrame
            table = StringIO(str(table_html))
            df = pd.read_html(table)[0]

            # Clean and transform the DataFrame
            try:
                df.drop(columns=6, inplace=True, errors='ignore')  # Safe column drop
                df.columns = df.iloc[0]
                df = df[2:]  # Drop the first 2 rows
                df = df.set_index(df.columns[0])
                df = df.dropna(how='all')
                df.reset_index(inplace=True)
                df = df.T
                df.columns = df.iloc[0]
                df = df[1:]
                df.index.name = 'Date'
                df.columns.name = None

            except Exception as e:
                print(f"Error processing table for {key}: {e}")
                continue

            # Write DataFrame to Excel
            df.to_excel(writer, sheet_name=key)
            print(f"{key} processed and added to {output_file}\n")

    return output_file


In [6]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
file_path = get_financials(links, company)
df = pd.read_excel(file_path, sheet_name='Ratios')
df

Processing Balance Sheet...
Balance Sheet processed and added to RelianceIndustries_Financials.xlsx

Processing Profit Loss...
Profit Loss processed and added to RelianceIndustries_Financials.xlsx

Processing Quarterly Results...
Quarterly Results processed and added to RelianceIndustries_Financials.xlsx

Processing Half Yearly Results...
Half Yearly Results processed and added to RelianceIndustries_Financials.xlsx

Processing Nine Months Results...
Nine Months Results processed and added to RelianceIndustries_Financials.xlsx

Processing Yearly Results...
Yearly Results processed and added to RelianceIndustries_Financials.xlsx

Processing Cash Flow...
Cash Flow processed and added to RelianceIndustries_Financials.xlsx

Processing Ratios...
Ratios processed and added to RelianceIndustries_Financials.xlsx



,Date,Basic EPS (Rs.),Diluted EPS (Rs.),Cash EPS (Rs.),Book Value [ExclRevalReserve]/Share (Rs.),Book Value [InclRevalReserve]/Share (Rs.),Dividend / Share(Rs.),Revenue from Operations/Share (Rs.),PBDIT/Share (Rs.),PBIT/Share (Rs.),...,Earnings Retention Ratio (%),Cash Earnings Retention Ratio (%),Enterprise Value (Cr.),EV/Net Operating Revenue (X),EV/EBITDA (X),MarketCap/Net Operating Revenue (X),Retention Ratios (%),Price/BV (X),Price/Net Operating Revenue,Earnings Yield
0,Mar 24,62.14,62.14,88.28,761.30,761.30,10.0,790.03,127.69,101.54,...,85.52,89.81,2156644.88,4.03,24.96,3.77,85.51,3.91,3.77,0.02
1,Mar 23,65.34,65.34,80.29,708.09,708.09,9.0,780.84,113.62,98.67,...,88.51,90.65,1736200.43,3.29,22.58,2.99,88.50,3.29,2.99,0.03
2,Mar 22,59.24,58.49,72.96,697.01,697.01,8.0,626.32,97.83,82.64,...,89.01,91.30,1954716.18,4.61,29.53,4.21,89.00,3.78,4.21,0.02
3,Mar 21,49.66,48.90,63.84,736.20,736.20,7.0,381.17,74.97,60.70,...,87.73,90.47,1479239.40,6.02,30.61,5.26,87.72,2.72,5.26,0.02
4,Mar 20,48.42,48.72,64.10,617.15,617.15,6.5,531.56,104.74,89.39,...,87.54,90.52,950998.06,2.82,14.32,2.09,87.53,1.80,2.09,0.04


In [ ]:
# def get_financials(links_dict = 'links', company = 'company'):

#     # Create an ExcelWriter to store DataFrames in an Excel file
#     output_file = f'{company}_Financials.xlsx'
#     with pd.ExcelWriter(output_file, engine='openpyxl') as writer:

#         for key, url in links_dict.items():
#             print(f"Processing {key}...")
#             # Get HTML data
#             response = requests.get(url)
#             html_data = response.content
#             # Parse the HTML
#             soup = BeautifulSoup(html_data, 'html.parser')
#             # Get table
#             table = StringIO(str(soup.find_all('table', class_='mctable1')))

#             # Conver to dataframe
#             df = pd.read_html(table)[0]  # Using pandas to read the HTML table
#             df.drop(columns=6, inplace=True)
#             df.columns = df.iloc[0]
#             df = df[2:]  # Drop the first 2 rows
#             df = df.set_index(df.columns[0])
#             df = df.dropna(how='all')
#             df.reset_index(inplace=True)
#             df = df.T
#             df.columns = df.iloc[0]
#             df = df[1:]
#             df.index.name = 'Date'
#             df.columns.name = None

#             # Write DataFrame to Excel
#             output = df.to_excel(writer, sheet_name=key)
#             print(" Done")
#     return output


In [8]:
# store data in a dataframe and store dataframes in excel file
output_file = f'{company}_Financials.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for key, url in links.items():
        print(f"Processing {key}...")
        # Get HTML data
        response = requests.get(url)
        html_data = response.content
        # Parse the HTML
        soup = BeautifulSoup(html_data, 'html.parser')
        # Get table
        table = StringIO(str(soup.find_all('table', class_='mctable1')))
        # Conver to dataframe
        df = pd.read_html(table)[0]  # Using pandas to read the HTML table
        df.drop(columns=6, inplace=True)
        df.columns = df.iloc[0]
        df = df[2:]  # Drop the first 2 rows
        df = df.set_index(df.columns[0])
        df = df.dropna(how='all')
        df.reset_index(inplace=True)
        df = df.T
        df.columns = df.iloc[0]
        df = df[1:]
        df.index.name = 'Date'
        df.columns.name = None

        # Write DataFrame to Excel
        df.to_excel(writer, sheet_name=key)
        print(" Done\n")

Processing Balance Sheet...
 Done

Processing Profit Loss...
 Done

Processing Quarterly Results...
 Done

Processing Half Yearly Results...
 Done

Processing Nine Months Results...
 Done

Processing Yearly Results...
 Done

Processing Cash Flow...
 Done

Processing Ratios...
 Done



In [9]:
df

,Basic EPS (Rs.),Diluted EPS (Rs.),Cash EPS (Rs.),Book Value [ExclRevalReserve]/Share (Rs.),Book Value [InclRevalReserve]/Share (Rs.),Dividend / Share(Rs.),Revenue from Operations/Share (Rs.),PBDIT/Share (Rs.),PBIT/Share (Rs.),PBT/Share (Rs.),...,Earnings Retention Ratio (%),Cash Earnings Retention Ratio (%),Enterprise Value (Cr.),EV/Net Operating Revenue (X),EV/EBITDA (X),MarketCap/Net Operating Revenue (X),Retention Ratios (%),Price/BV (X),Price/Net Operating Revenue,Earnings Yield
Date,,,,,,,,,,,,,,,,,,,,,
Mar 24,62.14,62.14,88.28,761.30,761.30,10.00,790.03,127.69,101.54,81.69,...,85.52,89.81,2156644.88,4.03,24.96,3.77,85.51,3.91,3.77,0.02
Mar 23,65.34,65.34,80.29,708.09,708.09,9.00,780.84,113.62,98.67,80.01,...,88.51,90.65,1736200.43,3.29,22.58,2.99,88.50,3.29,2.99,0.03
Mar 22,59.24,58.49,72.96,697.01,697.01,8.00,626.32,97.83,82.64,69.16,...,89.01,91.30,1954716.18,4.61,29.53,4.21,89.00,3.78,4.21,0.02
Mar 21,49.66,48.90,63.84,736.20,736.20,7.00,381.17,74.97,60.70,42.22,...,87.73,90.47,1479239.40,6.02,30.61,5.26,87.72,2.72,5.26,0.02
Mar 20,48.42,48.72,64.10,617.15,617.15,6.50,531.56,104.74,89.39,63.60,...,87.54,90.52,950998.06,2.82,14.32,2.09,87.53,1.80,2.09,0.04
